In [1]:
using Plots

using StaticArrays
import LinearAlgebra
import FastGaussQuadrature

push!(LOAD_PATH, pwd())
include("./physconsts.jl")
using .PhysConst


#-------------------------------------- STEP 1 -------------------------------------- #
# define necessary data structures

# module SpaceRadGrid

        
# gaussnodes -   Gauss-Legendre nodes
# gaussweights - Gauss-Legendre weights



using StaticArrays
import Base.@kwdef

    const IX = 1
    const IY = 2
    const IZ = 3

    @show const BIGNUMBER = 1.0E50


    const Rsc =1.0 #PhysConst.KM

    @kwdef struct SGridPar
        

        Nactive = 20;
    
        nghost = 1 #N of ghost cells at every boundary.
            
        Nx = Nactive;   # N of active cells
        Ny = Nactive;
        Nz = Nactive;
        
        Nxtot = Nx +  2*nghost;
        Nytot = Ny +  2*nghost;
        Nztot = Nz +  2*nghost;
    
        is = 1 + nghost; 
        ie = Nxtot -nghost;        

        js = 1 + nghost;  
        je = Nytot - nghost;
        
        ks = 1 + nghost;
        ke = Nztot - nghost;
            
        x1s = -3.0*Rsc
        x1e = 3.0*Rsc
    
        x2s = -3.0*Rsc
        x2e = 3.0*Rsc

        x3s = -1.0*Rsc
        x3e = 1.0*Rsc
        
    end

    @kwdef struct RGridPar
    
        Nfreq = 2 ; # ν - N bins 
    
        NumPhi = 5
        NumTheta = 5
    
        Nang = NumPhi*NumTheta    
        N_fre_ang = Nfreq*Nang  
            

    end




# ------ Init Space and Radiation grids --------------------------------

sg= SGridPar()
rad = RGridPar()


# radiaion matter grid *may be* offset from the space grid boundaries
roffst = 0

is_r = sg.is + roffst
ie_r = sg.ie - roffst 

js_r = sg.js + roffst
je_r = sg.je - roffst 

ks_r = sg.ks + roffst
ke_r = sg.ke - roffst 



# μ = zeros(Float64, rad.NumTheta)     
# @show gaussnodes, gaussweights = FastGaussQuadrature.gausslegendre(rad.NumTheta);
# @show μ .= gaussnodes

# @time nodes, weights = gausslegendre( 100000 );
# 0.007890 seconds (19 allocations: 2.671 MB)




θ = range(0, pi; length = rad.NumTheta)
ϕ   = range(0, 2pi-pi/rad.NumPhi; length = rad.NumPhi)


# ----------------------------------------------------------------------

#                                Grid

x1 = Array{Float64}(undef, sg.Nxtot)
x2 = Array{Float64}(undef, sg.Nytot)
x3 = Array{Float64}(undef, sg.Nztot)

x1b = Array{Float64}(undef, sg.Nxtot)
x2b = Array{Float64}(undef, sg.Nytot)
x3b = Array{Float64}(undef, sg.Nztot)

# ----------------------------------------------------------------------
#                               Gas   



den = zeros(Float64, sg.Nxtot,sg.Nytot,sg.Nztot) 
Tgas = zeros(Float64, sg.Nxtot,sg.Nytot,sg.Nztot) 

OpaCon = zeros(Float64, sg.Nxtot,sg.Nytot,sg.Nztot)


LW =1
UP =2
ndenIon = zeros(Float64, 2, sg.Nxtot,sg.Nytot,sg.Nztot) #2-level ion, cm^-3



# ----------------------------------------------------------------------
#                             Radiation
νRange = Array{Float64}(undef, rad.Nfreq) 

IrPos = zeros(Float64, rad.Nang, sg.Nxtot, sg.Nytot, sg.Nztot); # radiation intensity, forward
IrNeg = zeros(Float64, rad.Nang, sg.Nxtot, sg.Nytot, sg.Nztot); # radiation intensity, backward
Jν = zeros(Float64, rad.Nfreq, sg.Nxtot, sg.Nytot, sg.Nztot); # radiation mean intensity, J
Sν = zeros(Float64, rad.Nfreq, sg.Nxtot, sg.Nytot, sg.Nztot); #S_nu


# reference parameters

# const Tsc = 1.0E5  #a reference gas T, K
# const Dsc=10.0  #a reference gas ndens cm^-3


const lineId=1
# const ν_h1 = 3.29e15



const A_ion = 1 # ion atomic weight
const Nlines = 1
LineList = zeros(Float64, 5, Nlines)

iline =1

NU = 1
GSTAT_L = 2
GSTAT_U = 3
F_ORSC = 4
A_KL = 5
B_KL = 6

# O_VIII 1s-2p
LineList[NU,iline]      = 2.47e15    #nu or cm^-1, or λ
LineList[GSTAT_L,iline] = 2.0   #g_l
LineList[GSTAT_U,iline] = 6.0   #g_u
LineList[F_ORSC,iline]   = 4.16E-1 #f_lu
LineList[A_KL,iline]    = 2.57E12  #A_ki s^-1


# ν_min = 0.1*3.5e15     
# ν_max = 10*νLines[end]     

const NGaussHermNodes = 10

# ----------------------------------------------------------------------

function MakeXYZstagGrid(x1, x2, x3, x1b, x2b, x3b, sg)  #staggered grid

    # add check sizes
    
    xtmp = range(sg.x1s, sg.x1e; length = sg.Nxtot - 2sg.nghost)
    
#     @show sg.is, sg.ie sg.Nx sg.nghost size(xtmp) size(x1) size(sg.is:sg.ie) size(2:2)
    
    x1[sg.is:sg.ie] .= xtmp[:] 
    x1[sg.is-1] = 2*x1[sg.is]- x1[sg.is+1]
    x1[sg.ie+1] = 2*x1[sg.ie] -x1[sg.ie-1]

    xtmp = range(sg.x2s, sg.x2e; length = sg.Nytot - 2sg.nghost)    

    x2[sg.js:sg.je] .= xtmp[:]     
    x2[sg.js-1] = 2*x2[sg.js]- x2[sg.js+1]
    x2[sg.je+1] = 2*x2[sg.je] -x2[sg.je-1]

    xtmp = range(sg.x3s, sg.x3e; length = sg.Nztot - 2sg.nghost)    

    x3[sg.ks:sg.ke] .= xtmp[:]     
    x3[sg.ks-1] = 2*x3[sg.ks]- x3[sg.ks+1]
    x3[sg.ke+1] = 2*x3[sg.ke] -x3[sg.ke-1]
    
    
    x1b[1] = x1[1] - 0.5(x1[sg.is]-x1[1])    
    for i in sg.is:(sg.ie+1)
        x1b[i] = x1[i-1] + 0.5(x1[i]-x1[i-1])
    end

    x2b[1] = x2[1] - 0.5(x2[sg.js]-x2[1])    
    for j in sg.js:(sg.je+1)
        x2b[j] = x2[j-1] + 0.5(x2[j]-x2[j-1])
    end

    x3b[1] = x3[1] - 0.5(x3[sg.ks]-x3[1])    
    for k in sg.ks:(sg.ke+1)
        x3b[k] = x3[k-1] + 0.5(x3[k]-x3[k-1])
    end
    
     
end

mutable struct LongRay
    numOfElm::Int64    
    id::Int64 #id in dir array
    ijkOfCellCrossed :: Matrix{Int64}        
    xyzPos :: Matrix{Float64}
    orig::Int64 #the origin of the ray, as lin index
end



function InitRayDirections(rad::RGridPar, sg::SGridPar)
            
    norm = zeros(Float64, rad.Nang, 3);
    
    m=1
    for (i,ph_i) in enumerate(ϕ)
        for (j,th_j) in enumerate(θ)
            
            μj = cos.(th_j)
            
            sinth_j = sqrt(1-μj^2)
            

            norm[m, IX] = sinth_j *cos(ph_i)    
            
            norm[m, IY] = sinth_j *sin(ph_i)
            
            norm[m, IZ] = μj
#             @show norm[m,:] m            
        m+=1;            
        end
    end

    return(norm)
end

# initialize the angular discretization
norm = InitRayDirections(rad,sg)

    
# initialize the spacial grid    
MakeXYZstagGrid(x1,x2,x3,x1b,x2b,x3b,sg)


@show x3
@show x3b
 

    
NumElemInRayMax = 2*max(sg.Nxtot,sg.Nytot,sg.Nztot)

# NumElemInRayMax = sg.Nx+sg.Ny+sg.Nz   

# *****************************************************************
function PlotRay(ray::LongRay, ax1, ax2)

    

    @show y1 = ray.xyzPos[ax2,1:ray.numOfElm]
    @show x1 = ray.xyzPos[ax1,1:ray.numOfElm]
    
    p1 = plot(x1, y1,  seriestype = :scatter)
    p2 = plot!(p1, x1, y1)
        
    display(p2)

    
end 

function xyzToijk(xin::Float64, yin::Float64, zin::Float64)
    i = argmin( abs.(x1 .-xin )); 
    j = argmin(abs.(x2  .-yin)); 
    k = argmin(abs.(x3  .-zin))    
    return(i,j,k)
end

# argmin(νRange,1.0E14)

# *****************************************************************

#   Physics 

function LineDoppWidth(ν0::Float64, i::Int64, j::Int64, k::Int64 )
    @show Te::Float64 = Tgas[i,j,k]
    u0::Float64 = 12.85E+5*(Te/10^4/A_ion)^(1/2)         
    Δν_D::Float64 = u0*ν0/PhysConst.CL    
    
end

function LineDoppWidth(ν0::Float64, Te::Float64)
    u0::Float64 = 12.85E+5*(Te/10^4/A_ion)^(1/2)         
    Δν_D::Float64 = u0*ν0/PhysConst.CL        
end


using .PhysConst: HPL, KBLZ



# SPECTRUM
function LineProfDopp(x::Float64)    
    ϕ::Float64 = exp(-x^2)/sqrt(π)
end
function CrossSectionPhotoIonization(ν::Float64) #[cm^2]
    #photo-ionization cross-section

    Z=1.0    
    
    ν0 = 3.29e15
    
    if ν < ν0
        return(0.0)
    else
        
        ν1 = Z^2 * ν0     
        
        A0 = 6.3E-18
    
        y = ν/ν1
        
        ϵ = sqrt(y-1.0) 
    
        f1 = 1/(1.0 - exp(-2.0*pi/ϵ))
        
        e1 = exp(4.0 - 4.0*atan(ϵ)/ϵ)
           
        a = A0/Z^2*y^(-4)*e1*f1
    
        return(a)        
    end
    
end



@show Jν[:, is_r, js_r, ks_r]

@show nu = νRange[1]






@show  Δν_D = LineDoppWidth(LineList[1], is_r, ks_r, ks_r)

# LineProfDop(νLines[1], is_r, ks_r, ks_r)
# # integrates f(x) = x^2 from -1 to 1
# @time dot( weights, nodes.^2 )


# needed to integrate over the spec. l. profile



ξi, wi = FastGaussQuadrature.gausshermite(NGaussHermNodes);
@show ξi

@show LinearAlgebra.dot([1.0 for ξl in ξi], wi)/√π


ν_min = 0.5*LineList[NU,iline]
ν_max = 2*ν_min

# InitializeMatter(sg,rad)










const BIGNUMBER = 1.0e50 = 1.0e50
x3 = [-1.1052631578947367, -1.0, -0.8947368421052632, -0.7894736842105263, -0.6842105263157895, -0.5789473684210527, -0.47368421052631576, -0.3684210526315789, -0.2631578947368421, -0.15789473684210525, -0.05263157894736842, 0.05263157894736842, 0.15789473684210525, 0.2631578947368421, 0.3684210526315789, 0.47368421052631576, 0.5789473684210527, 0.6842105263157895, 0.7894736842105263, 0.8947368421052632, 1.0, 1.1052631578947367]
x3b = [-1.157894736842105, -1.0526315789473684, -0.9473684210526316, -0.8421052631578947, -0.736842105263158, -0.631578947368421, -0.5263157894736842, -0.42105263157894735, -0.3157894736842105, -0.21052631578947367, -0.10526315789473684, 0.0, 0.10526315789473684, 0.21052631578947367, 0.3157894736842105, 0.42105263157894735, 0.5263157894736842, 0.631578947368421, 0.736842105263158, 0.8421052631578947, 0.9473684210526316, 1.0526315789473684]
Jν[:, is_r, js_r, ks_r] = [0.0, 0.0]
nu = νRange[1] = 2.493965563e-314
Te::Float64 = Tgas

2.47e15

In [2]:
# module RadiatiomatTransfer
x3c = Array{Float64}(undef, 3);
xn = Array{Float64}(undef, 3)

distToNextCell = fill(BIGNUMBER,3) #used in FirstTimeTraceGridOverOneLongRay()
ijk_pos_s = Array{Int64,1}(undef,3)
ijk_pos = Array{Int64}(undef,3)

ijk_max = Array{Int64}(undef,3)
ijk_min = Array{Int64}(undef,3)
ijk_tmp = Array{Int64}(undef,3)

ijk_max .= [ sg.ie+1, sg.je+1, sg.ke+1 ] #max ijk index 
ijk_min .= [ sg.is-1, sg.js-1, sg.ks-1 ] #min index
ijk_tmp .= [0, 0, 0]



xbi_zip = [x1b, x2b, x3b]





function CheckForBoundary(xn, ijk_pos, posIndxToUpdate, iter, rbN)

    if iter > 100 
        error("iter too big in CheckForBoundary")
    end

#     @show iter , xn, rbN,x3b[sg.ke]
#         @show iter ,posIndxToUpdate, ijk_pos, ijk_max
    
    
#     if LinearAlgebra.norm(xn - rbN)<1.0E-6
# #             println(LinearAlgebra.norm(xn - rbN))
#         return(true)
#     end
    
    
    if sqrt((xn[1] - rbN[1])^2 + (xn[2] - rbN[2])^2 + (xn[3] - rbN[3])^2 )<1.0E-6
        return(true)
    end
    
    if ijk_pos[posIndxToUpdate] == ijk_max[posIndxToUpdate] ||     
        
            ijk_pos[posIndxToUpdate] == ijk_min[posIndxToUpdate] || 
        
           ijk_pos[posIndxToUpdate] == 1 ||                                    
            xn[1] <= x1b[sg.is] || xn[1] >= x1b[sg.ie+1] ||
            xn[2] <= x2b[sg.js] || xn[2] >= x2b[sg.je+1] ||
            xn[3] <= x3b[sg.ks] || xn[3] >= x3b[sg.ke+1]                        
#          println("quit..")
        return(true)
        
    else
        return(false)                
        end      

end

function GiveDistanceToNextCellBoundary(dir,iter,ijk_pos)
    """ starts from the boundary, passes through ray.head mid-cell, ends at the boundary"""
    
        BrakeOrNot=false
       
    for (n_it, norm_i ) in enumerate(dir) #QUESTION: maybe iter over pre-calclulated 1/norms

        
#         @show dir
        
           xc = x3c[n_it] 

             if iter >= 1 
             
             if norm_i != 0
               ijk_tmp[n_it] = ijk_pos[n_it] + copysign(1, dir[n_it])
             else
                ijk_tmp[n_it] = ijk_pos[n_it]
             end 
            
               itmp = ijk_tmp[n_it]; #@show iter , itmp, ijk_tmp
            
             if itmp == ijk_max[n_it] || itmp == ijk_min[n_it]   
                                
                BrakeOrNot = true
                                
                return(BrakeOrNot,BIGNUMBER)
                
             end
             
                xn[n_it] = xbi_zip[n_it][itmp]                                            

             end
                # @show itmp

             if norm_i != 0                      
                               
                distToNextCell[n_it] = abs((xn[n_it] - xc)/dir[n_it])

             else
                distToNextCell[n_it] = BIGNUMBER
             end

#         if  distToNextCell[n_it]==0
# #             @show xn[n_it], xc
#             @show dir,norm_i 
#                     println("iter= ",iter," xn=",  xn, " ===> ", "n_it = ",n_it," \n ",            
#                             "distToNextCell[",n_it,"]=",distToNextCell[n_it], "\n\n")
            
#             error("ad")
#         end

        end #for loop over possible directions


    
    
        return(BrakeOrNot,distToNextCell)
    end

function GetIntersectionXYZWithBoundary(dir,ijk_p)

    rbc = [0.0, 0.0 , 0.0]
    
#     display(rbc)
    
    Dist123 = [0.0, 0.0, 0.0]; 
    
        x3c .= [ x1[ijk_p[IX]], x2[ijk_p[IY]], x3[ijk_p[IZ]] ]
            
        for (i_123, norm_i ) in enumerate(dir)

            if norm_i != 0.0
                l1 = max(( xbc[1,i_123] .- x3c[i_123] )/norm_i,0.0)
                l2 = max(( xbc[2,i_123] .- x3c[i_123] )/norm_i,0.0)
                Dist123[i_123] = max(l1,l2)            

            else
                Dist123[i_123] = BIGNUMBER
            end

        end
        
        recId = argmin(Dist123)
    
 
        rbc .= x3c .+ dir * Dist123[recId]                
  
        i,j,k = xyzToijk(rbc[1],rbc[2],rbc[3])
    
#         @show rbc, x1[i], x2[j],x3[k]
    
#         ijk_pos[n_it] + copysign(1, dir[n_it])
    
    return(rbc, i,j,k)
end

# should go to module

GetIntersectionXYZWithBoundary (generic function with 1 method)

In [75]:
using .PhysConst: MP,ME,QE,CL,KPE,KM

function Bν(ν::Float64,T::Float64)
    Bnu = 2.0*PhysConst.HPL* ν^3/(PhysConst.CL^2)/
        (exp(PhysConst.HPL*ν/PhysConst.KBLZ/T)-1.) 

    Bnu = 1.  
    
  return (Bnu )
    
end

function Get_χ_ul(ν::Float64, i::Int64, j::Int64, k::Int64)
#     i -lower
#     j - upper
    
    Δν_D=LineDoppWidth(LineList[NU,lineId], Tgas[i,j,k])
    
    ϕ_x = 1.0    
    ϕ_ν = ϕ_x/Δν_D
        
    χ = π *QE^2/ME/CL * LineList[F_ORSC,lineId]*
    (ndenIon[LW,i,j,k] - LineList[GSTAT_L,lineId]*ndenIon[UP,i,j,k]/LineList[GSTAT_U,lineId])*ϕ_ν

    #     @show ndenIon[UP,i,j,k],ndenIon[LW,i,j,k]LineList[GSTAT_L,lineId],LineList[GSTAT_U,lineId], QE, CL, ME, Δν_D,ϕ_ν,χ

    if χ<0
        error("χ<0 in Get_χ_ul")        
    end
    
#     χ=0.4*6.0e14*PhysConst.MP

    if χ<1E-5
        @show "::::",LineList[F_ORSC,lineId], ndenIon[LW,i,j,k],i,j,k
    end    
    χ = KPE*den[i,j,k]*MP    
    χ *= Rsc    #non-dimensional  ̃χ ≡ R_sc χ
#     χ= abs(x3[k]-0.0)<0.3 ? 15.0 : 0
    χ= 5.0

    return (χ)
    
    
end

function SrcFun2lev(ν::Float64, i::Int64, j::Int64, k::Int64) # 2 level atom 

#     # ϵ::Float64, lineId::Int64, u::Float64, 
    
#     Te::Float64 =  Tgas[i,j,k]
            
#     Sf::Float64 = 0.0 #2.0PhysConst.HPL*ν^3/PhysConst.CL^2/((n_l*g_u/n_u/g_l)-1.0) 
            
#     Bp::Float64 = Bν(ν,Te) 
    
#     ϵ_th =  Giv_ϵ_th(lineId, i,j,k)
    
#     Jbar = Jν[lineId, i,j,k]
    
#     Sf = (1.0-ϵ_th)*Jbar + ϵ_th*Bp
    
# #     @show Sf    
   
# #     if x3[k] > (x3[sg.ks] + 0.5 )
# #         Sf = 0*((1.0-ϵ_th)*Jbar + ϵ_th*Bp)
# #     else
# #         Sf = (1.0-ϵ_th)*Jbar + ϵ_th*Bp
# # #         Sf*=0.0
# #     end
    
#     Sf = exp( -(x3[k])^2/0.1 )

    
#     Sf = abs(x3[k]-0.0)<0.3 ? (1.0-ϵ_th)*Jbar + ϵ_th*Bp : 0.0 
       
#     Sf = (1.0-ϵ_th)*Jbar + ϵ_th*Bp

#     Sf = Bp
    
    Sf=1.0
    return(Sf)
        
end

Nmat = NumElemInRayMax

χ_OnRay = zeros(Float64,NumElemInRayMax) #precalculated χ_line, on a ray

u = zeros(Float64, Nmat)
v = zeros(Float64, Nmat)
λii  = zeros(Float64, Nmat)        

dx_i = zeros(Float64, Nmat)

τn = zeros(Float64, Nmat)
A = zeros(Float64, Nmat)
B = zeros(Float64, Nmat)    
C = zeros(Float64, Nmat)
D = zeros(Float64, Nmat)
H = zeros(Float64, Nmat)
F = zeros(Float64, Nmat)
Z = zeros(Float64, Nmat)    
S = zeros(Float64, Nmat) #Rhs

function GetMatrixCoef(ν,ic,jc,kc,χ,dx_ip05,dx_im05,dxi,i,ikey)
    
    if i==1
        A[1] = 0. 
        B[1] = 1.0/dx_ip05 + χ/2.0    
        C[1] = 1.0/dx_ip05 - χ/2.0    
        S[1] = 0.0    
        H[1] = B[1] - C[1]        
        F[1] = H[1]/C[1]    
        Z[1] = S[1]/B[1]      
        
    elseif ikey==0 #in-between
        
        S[i] = χ_OnRay[i]^2*SrcFun2lev(ν, ic,jc,kc) 
        
        A[i] = 1.0 /(dxi*dx_im05)        
        B[i] = χ^2 + (1.0/dx_im05 + 1.0/dx_ip05)/dxi                
        C[i] = 1.0/(dxi*dx_ip05)  
        
        
        H[i] = -A[i] + B[i] - C[i]                                
        F[i]=(H[i]+ (A[i]*F[i-1])/(1.0+F[i-1]) )/C[i]        
        Z[i]=(S[i] + A[i]*Z[i-1] )/( 1.0 + F[i] )/C[i]      
        
        

    elseif ikey==-1
        #1st -order
        Ntot = i
        A[Ntot]= 0 
        C[Ntot] = 0.
        B[Ntot]=1.0/dx_im05+ χ/2.0
        S[Ntot] =0.0
        
        #2nd -order           
        A[Ntot]= 1.0/dx_im05
        B[Ntot]= 1.0/dx_im05 + 0.5*dx_im05*χ_OnRay[Nmat-1]^2                           
        S[Ntot] = 1/χ_OnRay[Nmat-1] + 0.5*dx_im05*χ_OnRay[Nmat-1]^2 *S[Nmat-1]
    end
    
#     @show i,":" A[i], B[i] ,C[i], S[i], H[i],F[i],Z[i]
    
#     return(a,b,c,s,h,f,z)
    
end 




function TraceRayAndFeautriSolve(dir::Vector{Float64}, rayOne::LongRay, 
        sg::SGridPar, ijk_pos, x3c, ijk_orig, rb1, rbN)
    
    """ trace ray along direction and solve RT with Feautrier method. 
        Notes: sometimes icreasing itermax may be necessary;
        x3c is needed as if start is not from cell-center """
   
#     future: precalc:
    minDistEst = ((x1[sg.ie]-x1[sg.is])^2 + (x2[sg.je]-x2[sg.js])^2 + (x3[sg.ke]-x3[sg.ks])^2)^1/2/(sg.Nxtot^2 + sg.Nytot^2 +sg.Nztot^2)^1/2

    ν = νRange[1]
    
    dist = dist_prev = 0.0   
    distToNextCell .= BIGNUMBER    
    dx_im05 = dx_ip05 = 0.0
    
    itermax = 3 * sg.Nx 
    
    Nr::Int64 = 0 
    
    
    xn .= x3c
                    
    if ijk_orig[1]==ijk_pos[1] && ijk_orig[2]==ijk_pos[2] && ijk_orig[3]==ijk_pos[3]

        rayOne.orig = 1
        
#         println(ijk_orig, "  ", ijk_pos)
#         error("ah!")
    end 

    rayOne.numOfElm = 1
    rayOne.ijkOfCellCrossed[:,1] .= ijk_pos
    rayOne.xyzPos[:,1] .= xn
    rayOne.orig = 1 #default 1elem ray
    
    posIndxToUpdate = zeros(Int64,3)

    skip=0  
    skip_max = itermax
    iter = 1 
    
    while iter <= itermax #max length, may be smaller than actual because of scipping
                      
        
        BrakeOrNot,distToNextCell = GiveDistanceToNextCellBoundary(dir,iter,ijk_pos)
      
        
        if BrakeOrNot
            break
        end
                        
        if distToNextCell[1] < distToNextCell[2] && distToNextCell[1] < distToNextCell[3]  && distToNextCell[1] > 0            
               posIndxToUpdate = 1
               ijk_pos[posIndxToUpdate] = ijk_tmp[posIndxToUpdate] #update only relevant one        
        elseif distToNextCell[2] < distToNextCell[1] && distToNextCell[2] < distToNextCell[3] && distToNextCell[2] > 0            
               posIndxToUpdate = 2            
               ijk_pos[posIndxToUpdate] = ijk_tmp[posIndxToUpdate] #update only relevant one    
        elseif distToNextCell[3] < distToNextCell[1] && distToNextCell[3] < distToNextCell[2]  && distToNextCell[3] > 0               
               posIndxToUpdate = 3            
               ijk_pos[posIndxToUpdate] = ijk_tmp[posIndxToUpdate] #update only relevant one        
        else
            posIndxToUpdate = argmin( distToNextCell )
        end
                                       
        
        dist = distToNextCell[posIndxToUpdate]       
        
        ijk_pos[posIndxToUpdate] = ijk_tmp[posIndxToUpdate] 
        
        
        xn .= x3c .+ dir*dist   
        x3c .= xn
        
        if dist > minDistEst                           
            
            rayOne.numOfElm = iter            
            rayOne.ijkOfCellCrossed[:,iter] .= ijk_pos                        
            rayOne.xyzPos[:,iter] .= xn            
                    
#             if ijk_orig[1]==ijk_pos[1] && ijk_orig[2]==ijk_pos[2] && ijk_orig[3]==ijk_pos[3]  
#                 # record the position along the ray when it crosses the cell current in ijk loop
#                 rayOne.orig = iter                
#             end 
                        
    
            
            # -- solver part 
            ic=ijk_pos[1]; jc=ijk_pos[2]; kc =ijk_pos[3]
            
            χ_OnRay[iter] = Get_χ_ul(ν,ic,jc,kc) 
            
            dx_im05 = dist_prev            
            dx_ip05 = dist
           
            dx_i[iter] = 0.5*(dx_im05+dx_ip05)
         
            if CheckForBoundary(xn,ijk_pos,posIndxToUpdate,iter,rbN) 
#                 println("CheckForBoundary")
                break                        
            end    
            Nr=iter
#              if iter>1                
#                 if dx_i[iter] < 1E-10 || dx_im05 < 1E-10 || dx_ip05< 1E-10
#                     @show "!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!"            
# #                     PlotRay(rayOne, 1, 3);
#                     error("attn: dx_i=0")
#                 end
#               end
            
        
            GetMatrixCoef(ν,ic,jc,kc,χ_OnRay[iter],dx_ip05,dx_im05,dx_i[iter],iter, 0)
            
            iter+=1
            dist_prev = dist
            
        else
            skip+=1 
#             @show skip,posIndxToUpdate
            
            if skip>skip_max
#                 @show skip,posIndxToUpdate
                error("skip>skip_max")
            end
        end #dist>0
        
                
    end #over ray
    
     if  Nr > 2 #minimum 3 for second order
    
#     going back with the susbstitution step

        dx_i[Nr]= 0.5(dx_i[Nr-1] + dx_i[Nr-2])    
    
#         dx_i[Nmat]=  xx[Nmat] .- xx[Nmat-1]    

        dx_im05 = dx_i[Nr]    
        dx_ip05 = dx_i[Nr]
           
        GetMatrixCoef(ν,ijk_pos[1],ijk_pos[2],ijk_pos[3],χ_OnRay[Nr],dx_ip05,dx_im05,dx_i[iter],Nr,-1)  

        u[Nr] = Z[Nr]
        v[Nr] = u[Nr]
        E_ip1 = 0.0
        for i = Nr-1:-1:2  #backsweep   
            u[i] = u[i+1]/(1+F[i]) + Z[i]
            τn[i] = τn[i+1] +  χ_OnRay[i]*dx_i[i]
            
            #@show dt = τn[i+1]-τn[i]         
            duds = (u[i+1]-u[i])/dx_i[i]
            v[i] = -1.0/χ_OnRay[i]*duds
            E_i = A[i]/(B[i]-C[i]*E_ip1)    #needed for Λ-operator 
            λii[i] = 1.0/((1.0-D[i]*E_ip1)*(B[i]-A[i]*D[i-1]))
            E_ip1 = E_i
        end 

        u[1] = u[2]/(1.0+F[1]) + Z[1]   
        v[1] = -u[1] 
    else
        u[1]=v[1]=0.0 #maybe better would be short characteristic in a one-cell RT
    end
        
     return(u,v,λii,τn,Nr)

end 




TraceRayAndFeautriSolve (generic function with 1 method)

In [77]:
xbc = [ [x1[sg.is], x1[sg.ie]] [x2[sg.js], x2[sg.je]] [x3[sg.ks], x3[sg.ke]]] #packed array of the domain bound. coords.

ijk_pos_head = [0,0,0] #init x,y,z coords of the current cell


function SolveRadTransfOnGrid()

# @time begin
    
is = is_r; ie = ie_r
js = js_r; je = je_r 
ks = ks_r; ke = ke_r     
    
posInRay=1
    
Nr = NumElemInRayMax
    
testRay = LongRay(Nr,1, Array{Int64}(undef,3,Nr),Array{Float64}(undef,3,Nr),posInRay) 
    
    
# @show ie,je,ke = is,js,ks =  xyzToijk( 0.0, 0.0, 0.0 ) 
    
#     for k=ks:ke, j=js:je, i=is:ie
 
    for k=ks:ke        
        for j=js:je        
            for i=is:ie    
  
                  ijk_pos_head.= [i,j,k]  #remember your origin
                

                    for m=1:rad.Nang
                        
#                     @show m = rand(1:rad.Nang)
            
                        dir = norm[m,:]                                                
                       
                        ijk_pos .= ijk_pos_head
            
                        rbN,ib,jb,kb = GetIntersectionXYZWithBoundary(dir, ijk_pos);      
            
                        rb1,ib,jb,kb = GetIntersectionXYZWithBoundary(-dir, ijk_pos);     

                        ijk_pos .= ijk_pos_s .= [ib, jb, kb]                                                                         
                        
                        x3c .= rb1 
                        u,v,λii,τn,Nr = TraceRayAndFeautriSolve(dir,testRay, sg, ijk_pos, x3c, ijk_pos_head, rb1, rbN) 
                    
                    
                    
#                     let
#                         ur = u[1:Nr]
#                         vr = v[1:Nr]
# #                         for i = 1:Nr
# #                             Ipos[i] = u[i] + v[i]                    
# #                             Ineg[i] = u[i] - v[i]           
# #                         end     
#                         s=1:Nr
                        
#                         plt = plot(layout = (1, 2))
#                         plot!(plt[1],s, ur,label = "u")
#                         plot!(plt[2],s, vr,label = "v")
                        
#                         display(plt)
#                         error("stop")
#                     end
                    
                        
            end # -m

                    
                    
                end                       
        end        
        @show k
    end    
        
#         end #i,j,k
    
    testRay=nothing
    
    println(" ************ done ************** \n")
    
end


@time SolveRadTransfOnGrid()

let
#     ray = arrOfLongRay[rand(1:rad.Nang),3,4,10]
#     PlotRay(ray, 3, 2)
end

k = 2
k = 3
k = 4
k = 5
k = 6
k = 7
k = 8
k = 9
k = 10
k = 11
k = 12
k = 13
k = 14
k = 15
k = 16
k = 17
k = 18
k = 19
k = 20
k = 21
 ************ done ************** 

 25.942598 seconds (559.35 M allocations: 9.650 GiB, 5.58% gc time)
